In [1]:
# Import required libraries
import numpy as np
import pandas as pd
from scipy.linalg import eig

# Display options
pd.set_option("display.float_format", "{:.4f}".format)
np.set_printoptions(precision=4, suppress=True)

In [2]:
# Define the problem structure
goal = "Select Best Strategic Supplier"
criteria = ["Cost", "Delivery", "Quality", "Innovation", "Sustainability"]
alternatives = ["GTS", "ISC", "RCI", "ETP"]

# All elements in order: Goal, Criteria, Alternatives
all_elements = [goal] + criteria + alternatives
n_total = len(all_elements)

print(f"Goal: {goal}")
print(f"Criteria: {criteria}")
print(f"Alternatives: {alternatives}")
print(f"Total elements: {n_total}")
print()

Goal: Select Best Strategic Supplier
Criteria: ['Cost', 'Delivery', 'Quality', 'Innovation', 'Sustainability']
Alternatives: ['GTS', 'ISC', 'RCI', 'ETP']
Total elements: 10



In [3]:
# Helper functions
def calculate_eigenvalue_priorities(matrix):
    """
    Calculate priorities using eigenvalue method
    """
    # Calculate eigenvalues and eigenvectors
    eigenvals, eigenvecs = eig(matrix)

    # Find principal eigenvalue (largest)
    principal_idx = np.argmax(eigenvals.real)
    principal_eigenval = eigenvals[principal_idx].real
    principal_eigenvec = eigenvecs[:, principal_idx].real

    # Normalize eigenvector to get priorities
    priorities = principal_eigenvec / np.sum(principal_eigenvec)

    return priorities, principal_eigenval


def calculate_consistency_ratio(matrix, eigenval):
    """
    Calculate consistency ratio for a comparison matrix
    """
    n = matrix.shape[0]

    # Random Index values (from Saaty's work)
    RI_dict = {
        1: 0.00,
        2: 0.00,
        3: 0.58,
        4: 0.90,
        5: 1.12,
        6: 1.24,
        7: 1.32,
        8: 1.41,
        9: 1.45,
        10: 1.49,
    }

    if n in RI_dict:
        RI = RI_dict[n]
    else:
        RI = 1.49  # For n > 10

    if n <= 2:
        return 0.0, 0.0  # Perfect consistency for n=1,2

    # Consistency Index
    CI = (eigenval - n) / (n - 1)

    # Consistency Ratio
    CR = CI / RI if RI > 0 else 0.0

    return CI, CR

In [4]:
def print_matrix_with_labels(matrix, labels, title):
    """
    Print a matrix with row and column labels
    """
    print(f"\n{title}:")
    print("-" * len(title))
    df = pd.DataFrame(matrix, index=labels, columns=labels)
    print(df.to_string(float_format="%.4f"))
    print()


def print_priorities(priorities, labels, title):
    """
    Print priorities with labels
    """
    print(f"{title}:")
    priorities_dict = {
        label: priority for label, priority in zip(labels, priorities)
    }
    print(f"Priorities: {priorities_dict}")
    return priorities_dict

In [ ]:
# Step 1: Criteria Interdependency Matrix

# Matrix: Criteria vs Criteria (5x5)
criteria_interdependency = np.array(
    [
        #      Cost    Delivery  Quality  Innovation  Sustainability
        [1.0, 3.0, 1 / 3, 5.0, 7.0],  # Cost
        [1 / 3, 1.0, 1 / 5, 3.0, 5.0],  # Delivery
        [3.0, 5.0, 1.0, 7.0, 9.0],  # Quality
        [1 / 5, 1 / 3, 1 / 7, 1.0, 2.0],  # Innovation
        [1 / 7, 1 / 5, 1 / 9, 1 / 2, 1.0],  # Sustainability
    ]
)

print("Step 1: Criteria Interdependency Matrix")
print("Question: Which criterion has more influence on achieving the goal?")
print_matrix_with_labels(
    criteria_interdependency, criteria, "Criteria Interdependency Matrix"
)

# Calculate priorities and check consistency
priorities_criteria, max_eig_criteria = calculate_eigenvalue_priorities(
    criteria_interdependency
)
ci_criteria, cr_criteria = calculate_consistency_ratio(
    criteria_interdependency, max_eig_criteria
)

print_priorities(priorities_criteria, criteria, "Criteria Priorities")
print(f"CR = {cr_criteria:.4f}")
print(
    f"Consistency: {'Acceptable' if cr_criteria <= 0.10 else 'Needs revision'}"
)
print("=" * 50)

Step 1: Criteria Interdependency Matrix
Question: Which criterion has more influence on achieving the goal?

Criteria Interdependency Matrix:
-------------------------------
                 Cost  Delivery  Quality  Innovation  Sustainability
Cost           1.0000    3.0000   0.3333      5.0000          7.0000
Delivery       0.3333    1.0000   0.2000      3.0000          5.0000
Quality        3.0000    5.0000   1.0000      7.0000          9.0000
Innovation     0.2000    0.3333   0.1429      1.0000          2.0000
Sustainability 0.1429    0.2000   0.1111      0.5000          1.0000

Criteria Priorities:
Priorities: {'Cost': np.float64(0.2621387213763264), 'Delivery': np.float64(0.12934807910483373), 'Quality': np.float64(0.5153543243312695), 'Innovation': np.float64(0.05747738736035494), 'Sustainability': np.float64(0.03568148782721546)}
CR = 0.0412
Consistency: Acceptable


In [6]:
# Step 2: Alternatives Comparison Matrices

# Alternatives comparison with respect to Cost
alt_cost = np.array(
    [
        #     GTS   ISC   RCI   ETP
        [1.0, 3.0, 5.0, 7.0],  # GTS
        [1 / 3, 1.0, 3.0, 5.0],  # ISC
        [1 / 5, 1 / 3, 1.0, 3.0],  # RCI
        [1 / 7, 1 / 5, 1 / 3, 1.0],  # ETP
    ]
)

# Alternatives comparison with respect to Delivery
alt_delivery = np.array(
    [
        #     GTS   ISC   RCI   ETP
        [1.0, 1 / 3, 3.0, 5.0],  # GTS
        [3.0, 1.0, 5.0, 7.0],  # ISC
        [1 / 3, 1 / 5, 1.0, 3.0],  # RCI
        [1 / 5, 1 / 7, 1 / 3, 1.0],  # ETP
    ]
)

# Alternatives comparison with respect to Quality
alt_quality = np.array(
    [
        #     GTS   ISC   RCI   ETP
        [1.0, 3.0, 1 / 3, 5.0],  # GTS
        [1 / 3, 1.0, 1 / 5, 3.0],  # ISC
        [3.0, 5.0, 1.0, 7.0],  # RCI
        [1 / 5, 1 / 3, 1 / 7, 1.0],  # ETP
    ]
)

# Alternatives comparison with respect to Innovation
alt_innovation = np.array(
    [
        #     GTS   ISC   RCI   ETP
        [1.0, 1 / 5, 3.0, 1 / 3],  # GTS
        [5.0, 1.0, 7.0, 3.0],  # ISC
        [1 / 3, 1 / 7, 1.0, 1 / 5],  # RCI
        [3.0, 1 / 3, 5.0, 1.0],  # ETP
    ]
)

# Alternatives comparison with respect to Sustainability
alt_sustainability = np.array(
    [
        #     GTS   ISC   RCI   ETP
        [1.0, 3.0, 1 / 5, 1 / 7],  # GTS
        [1 / 3, 1.0, 1 / 7, 1 / 9],  # ISC
        [5.0, 7.0, 1.0, 1 / 3],  # RCI
        [7.0, 9.0, 3.0, 1.0],  # ETP
    ]
)

# Calculate priorities for each comparison
print("Step 2: Alternatives Comparison Matrices")
print("=" * 45)

# Cost
print_matrix_with_labels(alt_cost, alternatives, "Alternatives vs Cost")
priorities_cost, max_eig_cost = calculate_eigenvalue_priorities(alt_cost)
ci_cost, cr_cost = calculate_consistency_ratio(alt_cost, max_eig_cost)
print_priorities(priorities_cost, alternatives, "Cost Priorities")
print(f"CR = {cr_cost:.4f}")
print()

# Delivery
print_matrix_with_labels(
    alt_delivery, alternatives, "Alternatives vs Delivery"
)
priorities_delivery, max_eig_delivery = calculate_eigenvalue_priorities(
    alt_delivery
)
ci_delivery, cr_delivery = calculate_consistency_ratio(
    alt_delivery, max_eig_delivery
)
print_priorities(priorities_delivery, alternatives, "Delivery Priorities")
print(f"CR = {cr_delivery:.4f}")
print()

# Quality
print_matrix_with_labels(alt_quality, alternatives, "Alternatives vs Quality")
priorities_quality, max_eig_quality = calculate_eigenvalue_priorities(
    alt_quality
)
ci_quality, cr_quality = calculate_consistency_ratio(
    alt_quality, max_eig_quality
)
print_priorities(priorities_quality, alternatives, "Quality Priorities")
print(f"CR = {cr_quality:.4f}")
print()

# Innovation
print_matrix_with_labels(
    alt_innovation, alternatives, "Alternatives vs Innovation"
)
priorities_innovation, max_eig_innovation = calculate_eigenvalue_priorities(
    alt_innovation
)
ci_innovation, cr_innovation = calculate_consistency_ratio(
    alt_innovation, max_eig_innovation
)
print_priorities(priorities_innovation, alternatives, "Innovation Priorities")
print(f"CR = {cr_innovation:.4f}")
print()

# Sustainability
print_matrix_with_labels(
    alt_sustainability, alternatives, "Alternatives vs Sustainability"
)
priorities_sustainability, max_eig_sustainability = (
    calculate_eigenvalue_priorities(alt_sustainability)
)
ci_sustainability, cr_sustainability = calculate_consistency_ratio(
    alt_sustainability, max_eig_sustainability
)
print_priorities(
    priorities_sustainability, alternatives, "Sustainability Priorities"
)
print(f"CR = {cr_sustainability:.4f}")
print("=" * 50)

Step 2: Alternatives Comparison Matrices

Alternatives vs Cost:
--------------------
       GTS    ISC    RCI    ETP
GTS 1.0000 3.0000 5.0000 7.0000
ISC 0.3333 1.0000 3.0000 5.0000
RCI 0.2000 0.3333 1.0000 3.0000
ETP 0.1429 0.2000 0.3333 1.0000

Cost Priorities:
Priorities: {'GTS': np.float64(0.5650090537990473), 'ISC': np.float64(0.26220120556087323), 'RCI': np.float64(0.11750425004139858), 'ETP': np.float64(0.05528549059868092)}
CR = 0.0433


Alternatives vs Delivery:
------------------------
       GTS    ISC    RCI    ETP
GTS 1.0000 0.3333 3.0000 5.0000
ISC 3.0000 1.0000 5.0000 7.0000
RCI 0.3333 0.2000 1.0000 3.0000
ETP 0.2000 0.1429 0.3333 1.0000

Delivery Priorities:
Priorities: {'GTS': np.float64(0.26220120556087323), 'ISC': np.float64(0.5650090537990472), 'RCI': np.float64(0.11750425004139861), 'ETP': np.float64(0.055285490598680924)}
CR = 0.0433


Alternatives vs Quality:
-----------------------
       GTS    ISC    RCI    ETP
GTS 1.0000 3.0000 0.3333 5.0000
ISC 0.3333 1.0000 

In [7]:
# Step 3: Alternatives Interdependency

# How alternatives influence each other (market positioning, partnerships)
alt_interdependency = np.array(
    [
        #     GTS   ISC   RCI   ETP
        [1.0, 2.0, 1 / 3, 3.0],  # GTS
        [1 / 2, 1.0, 1 / 5, 2.0],  # ISC
        [3.0, 5.0, 1.0, 7.0],  # RCI
        [1 / 3, 1 / 2, 1 / 7, 1.0],  # ETP
    ]
)

print("Step 3: Alternatives Interdependency Matrix")
print(
    "Question: Which supplier has more influence on the others (market positioning, partnerships)?"
)
print_matrix_with_labels(
    alt_interdependency, alternatives, "Alternatives Interdependency"
)

priorities_alt_inter, max_eig_alt = calculate_eigenvalue_priorities(
    alt_interdependency
)
ci_alt, cr_alt = calculate_consistency_ratio(alt_interdependency, max_eig_alt)

print_priorities(
    priorities_alt_inter,
    alternatives,
    "Alternatives Interdependency Priorities",
)
print(f"CR = {cr_alt:.4f}")
print("=" * 50)

Step 3: Alternatives Interdependency Matrix
Question: Which supplier has more influence on the others (market positioning, partnerships)?

Alternatives Interdependency:
----------------------------
       GTS    ISC    RCI    ETP
GTS 1.0000 2.0000 0.3333 3.0000
ISC 0.5000 1.0000 0.2000 2.0000
RCI 3.0000 5.0000 1.0000 7.0000
ETP 0.3333 0.5000 0.1429 1.0000

Alternatives Interdependency Priorities:
Priorities: {'GTS': np.float64(0.21787642541536487), 'ISC': np.float64(0.12278629071088788), 'RCI': np.float64(0.58716568217102), 'ETP': np.float64(0.07217160170272717)}
CR = 0.0071


In [8]:
# Step 4: Cluster Comparison

# Define clusters: Goal, Criteria, Alternatives
cluster_labels = ["Goal", "Criteria", "Alternatives"]

# Cluster comparison matrix - which cluster is more important for the network
# In ANP, we typically focus on how criteria and alternatives interact
cluster_comparison = np.array(
    [
        #   Goal  Criteria  Alternatives
        [0.0, 0.0, 0.0],  # Goal - doesn't influence others
        [1.0, 1.0, 2.0],  # Criteria - more important than alternatives
        [0.1, 0.5, 1.0],  # Alternatives - less important than criteria
    ]
)

print("Step 4: Cluster Comparison Matrix")
print("Question: Which cluster has more influence in the network?")
print_matrix_with_labels(
    cluster_comparison, cluster_labels, "Cluster Comparison Matrix"
)

# Calculate cluster weights (normalized by column)
column_sums = np.sum(cluster_comparison, axis=0)
cluster_weights = {}
for i, cluster in enumerate(cluster_labels):
    if column_sums[i] > 0:
        cluster_weights[cluster] = cluster_comparison[:, i] / column_sums[i]
    else:
        cluster_weights[cluster] = np.zeros(3)

print("Cluster Weights (normalized by column):")
for cluster, weights in cluster_weights.items():
    print(f"{cluster}: {weights}")

print("=" * 50)

Step 4: Cluster Comparison Matrix
Question: Which cluster has more influence in the network?

Cluster Comparison Matrix:
-------------------------
               Goal  Criteria  Alternatives
Goal         0.0000    0.0000        0.0000
Criteria     1.0000    1.0000        2.0000
Alternatives 0.1000    0.5000        1.0000

Cluster Weights (normalized by column):
Goal: [0.     0.9091 0.0909]
Criteria: [0.     0.6667 0.3333]
Alternatives: [0.     0.6667 0.3333]


In [9]:
# Step 5: Construct Unweighted Supermatrix

# Initialize 10x10 supermatrix (Goal + 5 Criteria + 4 Alternatives)
supermatrix = np.zeros((n_total, n_total))

# Goal row remains zeros (Goal doesn't influence anything - it's the target)
# Goal column will be filled with relationships TO the goal

# Criteria relationships (rows 1-5, columns 1-5)
# How criteria influence each other
supermatrix[1:6, 1:6] = criteria_interdependency

# Alternatives to Criteria relationships (rows 6-9, columns 1-5)
# How alternatives relate to criteria (using calculated priorities)
supermatrix[6:10, 1] = priorities_cost  # Cost
supermatrix[6:10, 2] = priorities_delivery  # Delivery
supermatrix[6:10, 3] = priorities_quality  # Quality
supermatrix[6:10, 4] = priorities_innovation  # Innovation
supermatrix[6:10, 5] = priorities_sustainability  # Sustainability

# Alternatives interdependencies (rows 6-9, columns 6-9)
supermatrix[6:10, 6:10] = alt_interdependency

# Criteria to Goal relationships (rows 1-5, column 0)
# How criteria contribute to the goal
supermatrix[1:6, 0] = priorities_criteria

# Alternatives to Goal relationships (rows 6-9, column 0)
# Direct contribution of alternatives to goal (often small or zero in ANP)
supermatrix[6:10, 0] = [0.1, 0.1, 0.1, 0.1]  # Small direct contribution

# Ensure Goal row is all zeros (Goal doesn't influence other elements)
supermatrix[0, :] = 0

print("Step 5: Unweighted Supermatrix")
print("=" * 40)

# Display supermatrix with labels
supermatrix_df = pd.DataFrame(
    supermatrix, index=all_elements, columns=all_elements
)
supermatrix_df

Step 5: Unweighted Supermatrix


,Select Best Strategic Supplier,Cost,Delivery,Quality,Innovation,Sustainability,GTS,ISC,RCI,ETP
Select Best Strategic Supplier,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
Cost,0.2621,1.0000,3.0000,0.3333,5.0000,7.0000,0.0000,0.0000,0.0000,0.0000
Delivery,0.1293,0.3333,1.0000,0.2000,3.0000,5.0000,0.0000,0.0000,0.0000,0.0000
Quality,0.5154,3.0000,5.0000,1.0000,7.0000,9.0000,0.0000,0.0000,0.0000,0.0000
Innovation,0.0575,0.2000,0.3333,0.1429,1.0000,2.0000,0.0000,0.0000,0.0000,0.0000
Sustainability,0.0357,0.1429,0.2000,0.1111,0.5000,1.0000,0.0000,0.0000,0.0000,0.0000
GTS,0.1000,0.5650,0.2622,0.2622,0.1175,0.0849,1.0000,2.0000,0.3333,3.0000
ISC,0.1000,0.2622,0.5650,0.1175,0.5650,0.0425,0.5000,1.0000,0.2000,2.0000
RCI,0.1000,0.1175,0.1175,0.5650,0.0553,0.2895,3.0000,5.0000,1.0000,7.0000
ETP,0.1000,0.0553,0.0553,0.0553,0.2622,0.5831,0.3333,0.5000,0.1429,1.0000


In [10]:
# Step 6: Create Weighted Supermatrix

# Extract calculated cluster weights from Step 4
criteria_cluster_weight = cluster_weights["Criteria"][
    1
]  # How criteria influence criteria block
alternatives_cluster_weight = cluster_weights["Alternatives"][
    2
]  # How alternatives influence alternatives block

# Create weighted supermatrix using calculated weights
weighted_supermatrix = supermatrix.copy()

# Apply cluster weights to respective blocks
# Column 0 (Goal): No weighting needed - it's the target
# Columns 1-5 (Criteria): Weight by criteria cluster importance
# Columns 6-9 (Alternatives): Weight by alternatives cluster importance
weighted_supermatrix[:, 1:6] *= criteria_cluster_weight
weighted_supermatrix[:, 6:10] *= alternatives_cluster_weight

# Normalize columns to be stochastic (column sums = 1)
for j in range(n_total):
    col_sum = np.sum(weighted_supermatrix[:, j])
    if col_sum > 0:
        weighted_supermatrix[:, j] = weighted_supermatrix[:, j] / col_sum

print("Step 6: Weighted Supermatrix")
print("=" * 35)

# Display weighted supermatrix
weighted_supermatrix_df = pd.DataFrame(
    weighted_supermatrix, index=all_elements, columns=all_elements
)
weighted_supermatrix_df

Step 6: Weighted Supermatrix


,Select Best Strategic Supplier,Cost,Delivery,Quality,Innovation,Sustainability,GTS,ISC,RCI,ETP
Select Best Strategic Supplier,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
Cost,0.1872,0.1762,0.2848,0.1196,0.2857,0.2800,0.0000,0.0000,0.0000,0.0000
Delivery,0.0924,0.0587,0.0949,0.0718,0.1714,0.2000,0.0000,0.0000,0.0000,0.0000
Quality,0.3681,0.5285,0.4747,0.3588,0.4000,0.3600,0.0000,0.0000,0.0000,0.0000
Innovation,0.0411,0.0352,0.0316,0.0513,0.0571,0.0800,0.0000,0.0000,0.0000,0.0000
Sustainability,0.0255,0.0252,0.0190,0.0399,0.0286,0.0400,0.0000,0.0000,0.0000,0.0000
GTS,0.0714,0.0995,0.0249,0.0941,0.0067,0.0034,0.2069,0.2353,0.1989,0.2308
ISC,0.0714,0.0462,0.0536,0.0422,0.0323,0.0017,0.1034,0.1176,0.1193,0.1538
RCI,0.0714,0.0207,0.0112,0.2027,0.0032,0.0116,0.6207,0.5882,0.5966,0.5385
ETP,0.0714,0.0097,0.0052,0.0198,0.0150,0.0233,0.0690,0.0588,0.0852,0.0769


In [11]:
# Step 7: Calculate Limit Matrix

# The limit matrix is computed by raising the weighted supermatrix to a large power
# This converges to the long-term influence priorities


def calculate_limit_matrix(matrix, max_iterations=100, tolerance=1e-6):
    """
    Calculate limit matrix by repeated multiplication
    """
    current_matrix = matrix.copy()

    for i in range(max_iterations):
        next_matrix = np.linalg.matrix_power(current_matrix, 2)

        # Check convergence
        if np.allclose(current_matrix, next_matrix, atol=tolerance):
            print(f"Converged after {i + 1} iterations")
            break

        current_matrix = next_matrix

    return current_matrix


print("Step 7: Calculate Limit Matrix")

# Calculate limit matrix
limit_matrix = calculate_limit_matrix(weighted_supermatrix)

# Display limit matrix with labels
limit_matrix_df = pd.DataFrame(
    limit_matrix, index=all_elements, columns=all_elements
)
print("\nLimit Matrix:")
print(limit_matrix_df.to_string(float_format="%.4f"))

# Extract final priorities from Goal column (column 0)
final_priorities = limit_matrix[:, 0]

print("\nFinal Priorities (from Goal column):")
for i, element in enumerate(all_elements):
    print(f"{element:<20}: {final_priorities[i]:.4f}")

Step 7: Calculate Limit Matrix
Converged after 7 iterations

Limit Matrix:
                                Select Best Strategic Supplier   Cost  Delivery  Quality  Innovation  Sustainability    GTS    ISC    RCI    ETP
Select Best Strategic Supplier                          0.0000 0.0000    0.0000   0.0000      0.0000          0.0000 0.0000 0.0000 0.0000 0.0000
Cost                                                    0.0000 0.0000    0.0000   0.0000      0.0000          0.0000 0.0000 0.0000 0.0000 0.0000
Delivery                                                0.0000 0.0000    0.0000   0.0000      0.0000          0.0000 0.0000 0.0000 0.0000 0.0000
Quality                                                 0.0000 0.0000    0.0000   0.0000      0.0000          0.0000 0.0000 0.0000 0.0000 0.0000
Innovation                                              0.0000 0.0000    0.0000   0.0000      0.0000          0.0000 0.0000 0.0000 0.0000 0.0000
Sustainability                                         

In [12]:
# Step 8: Final Results and Analysis

print("Step 8: Final Results and Decision Analysis")

# Extract only alternatives priorities
alternatives_priorities = final_priorities[
    6:10
]  # Last 4 elements are alternatives

# Create results summary
results_df = pd.DataFrame(
    {
        "Alternative": alternatives,
        "Priority": alternatives_priorities,
        "Percentage": alternatives_priorities * 100,
        "Rank": range(1, len(alternatives) + 1),
    }
)

# Sort by priority to get proper rankings
results_df = results_df.sort_values("Priority", ascending=False).reset_index(
    drop=True
)
results_df["Rank"] = range(1, len(alternatives) + 1)

print("\nFINAL RANKING:")
print(results_df.to_string(index=False, float_format="%.4f"))

# Best alternative
best_alternative = results_df.iloc[0]["Alternative"]
best_score = results_df.iloc[0]["Priority"]

print("\nRECOMMENDATION:")
print(f"Best supplier: {best_alternative}")
print(f"Priority score: {best_score:.4f} ({best_score * 100:.1f}%)")

# Show individual criteria performance
print("\nDETAILED ANALYSIS:")

print("\nCriteria Priorities (from Step 1):")
for i, criterion in enumerate(criteria):
    print(f"  {criterion:<15}: {priorities_criteria[i]:.4f}")

print("\nSupplier Performance by Criteria:")
print(f"{'Criterion':<15} {'GTS':<8} {'ISC':<8} {'RCI':<8} {'ETP':<8}")
print("-" * 55)
print(
    f"{'Cost':<15} {priorities_cost[0]:<8.4f} {priorities_cost[1]:<8.4f} {priorities_cost[2]:<8.4f} {priorities_cost[3]:<8.4f}"
)
print(
    f"{'Delivery':<15} {priorities_delivery[0]:<8.4f} {priorities_delivery[1]:<8.4f} {priorities_delivery[2]:<8.4f} {priorities_delivery[3]:<8.4f}"
)
print(
    f"{'Quality':<15} {priorities_quality[0]:<8.4f} {priorities_quality[1]:<8.4f} {priorities_quality[2]:<8.4f} {priorities_quality[3]:<8.4f}"
)
print(
    f"{'Innovation':<15} {priorities_innovation[0]:<8.4f} {priorities_innovation[1]:<8.4f} {priorities_innovation[2]:<8.4f} {priorities_innovation[3]:<8.4f}"
)
print(
    f"{'Sustainability':<15} {priorities_sustainability[0]:<8.4f} {priorities_sustainability[1]:<8.4f} {priorities_sustainability[2]:<8.4f} {priorities_sustainability[3]:<8.4f}"
)

print("\nNetwork Effects (Interdependencies):")
print(f"{'Supplier':<12} {'Influence Score':<15}")
print("-" * 27)
for i, supplier in enumerate(alternatives):
    print(f"{supplier:<12} {priorities_alt_inter[i]:<15.4f}")

print("\nKey Insights:")
print(
    f"1. {criteria[np.argmax(priorities_criteria)]} is the most important criterion ({max(priorities_criteria):.1%})"
)
print(
    f"2. {alternatives[np.argmax(priorities_alt_inter)]} has strongest network influence ({max(priorities_alt_inter):.1%})"
)
print("3. ANP considers both direct performance and network effects")
print(
    "4. Final decision incorporates interdependencies between criteria and alternatives"
)
print("5. Quality and cost are major decision factors in supplier selection")

Step 8: Final Results and Decision Analysis

FINAL RANKING:
Alternative  Priority  Percentage  Rank
        RCI    0.5961     59.6059     1
        GTS    0.2073     20.7338     2
        ISC    0.1185     11.8526     3
        ETP    0.0781      7.8078     4

RECOMMENDATION:
Best supplier: RCI
Priority score: 0.5961 (59.6%)

DETAILED ANALYSIS:

Criteria Priorities (from Step 1):
  Cost           : 0.2621
  Delivery       : 0.1293
  Quality        : 0.5154
  Innovation     : 0.0575
  Sustainability : 0.0357

Supplier Performance by Criteria:
Criterion       GTS      ISC      RCI      ETP     
-------------------------------------------------------
Cost            0.5650   0.2622   0.1175   0.0553  
Delivery        0.2622   0.5650   0.1175   0.0553  
Quality         0.2622   0.1175   0.5650   0.0553  
Innovation      0.1175   0.5650   0.0553   0.2622  
Sustainability  0.0849   0.0425   0.2895   0.5831  

Network Effects (Interdependencies):
Supplier     Influence Score
-----------------